## Download html files

request,bsでは動的コンテンツをとってこれないのでseleniumを使う

In [ ]:
from selenium import webdriver
from time import sleep
import requests

In [3]:
url="https://toho-vote.info/result"

In [ ]:
driver = webdriver.Chrome('ChromeDriverのディレクトリ + chromedriver')

In [5]:
driver.get(url)
for button in []:
    driver.find_element_by_xpath(button).click()
    time.sleep(1)

<Response [200]>

In [ ]:
driver.quit()

## htmlからcsvファイル作成(scraping)

In [1]:
from bs4 import BeautifulSoup  # BeautifulSoupクラスをインポート
ddir="data"

In [5]:
filename_tr="/18/第18回東方Project人気投票_人妖"
filename=ddir+filename_tr+".html"

In [5]:
with open(ddir+filename,"r") as fp:
    htmltext=fp.read()
soup = BeautifulSoup(htmltext, "html.parser")

ブラウザのデバッガ
document.querySelector("#app > div > main > div._container_1r0po_1._root_1orw9_14 > div > div > div > table")
の結果から

In [4]:
soup.body.div.div.main.find("div", class_="_root_1orw9_14").div.div

<div class="_title_en_1h8e9_51">Characters</div>

In [5]:
soup.body.div.div.main.find("div", class_="_root_1orw9_14").div.header

<header class="_title_1h8e9_1"><h1>人妖部門</h1><div class="_title_en_1h8e9_51">Characters</div></header>

In [6]:
def get_tabledata(table):
    return [[td.string for td in tr.find_all("td")] for tr in table]

In [7]:
table=soup.body.div.div.main.find("div", class_="_root_1orw9_14").div


In [8]:
table_header=table.find_all("th")
table_contents=table.find_all("tr")

In [9]:
table_header

[<th>順位</th>,
 <th class="_rank_prev_1orw9_133">前回</th>,
 <th class="_rank_prev2_1orw9_136">前々</th>,
 <th class="_name_1orw9_101">名前</th>,
 <th>ポイント</th>,
 <th>一推し</th>,
 <th>コメント</th>,
 <th>支援作品</th>]

In [12]:
outdata=get_tabledata(table_contents)

In [14]:
outdata[0]=outheader
outdata[:2]

[[], ['1', '5', '5', 'フランドール・スカーレット', '16,723', '3,730', '3,633', '19']]

### output

## 第16回以前の投票結果

文字コード

In [85]:
!nkf -g */*/*.html

data/1/東方シリーズキャラ人気投票 [Vote Top].html: CP932
data/1/東方シリーズ音楽人気投票 [Vote Top].html: CP932
data/1/東方シリーズ音楽人気投票 [Vote Top]_iro.html: CP932
data/10/第１０回東方シリーズ人気投票 キャラ部門集計結果.html: EUC-JP
data/10/第１０回東方シリーズ人気投票 音楽部門集計結果.html: EUC-JP
data/10/第１０回東方シリーズ人気投票 投票者アンケート結果.html: EUC-JP
data/11/アンケート投票結果 - 第11回 東方Project 人気投票.html: UTF-8
data/11/キャラ部門投票結果 - 第11回 東方Project 人気投票.html: UTF-8
data/11/音楽部門投票結果 - 第11回 東方Project 人気投票.html: UTF-8
data/11/経緯報告書 - 第11回 東方Project 人気投票.html: UTF-8
data/12/アンケート部門集計結果 - 第12回 東方Project 人気投票.html: UTF-8
data/12/音楽部門投票結果 - 第12回 東方Project 人気投票.html: UTF-8
data/12/作品部門投票結果 - 第12回 東方Project 人気投票.html: UTF-8
data/12/人妖部門投票結果 - 第12回 東方Project 人気投票.html: UTF-8
data/13/アンケート集計結果 - 第13回 東方Project 人気投票.html: UTF-8
data/13/ベストパートナー部門投票結果 - 第13回 東方Project 人気投票.html: UTF-8
data/13/音楽部門投票結果 - 第13回 東方Project 人気投票.html: UTF-8
data/13/作品部門投票結果 - 第13回 東方Project 人気投票.html: UTF-8
data/13/人妖部門投票結果 - 第13回 東方Project 人気投票.html: UTF-8
data/14/アンケート部門集計結果 - 第14回 東方Project 人気投票.html: UTF-8
da

UTF-8, EUC＿JP,CP932が混在している

In [38]:
import csv

def get_tabledata(table):
    return [[t.string for t in tr.find_all("td")]+[t.string for t in tr.find_all("th")] for tr in table]

def get_tableheader(table):
    return [[t.string for t in tr.find_all("th")] for tr in table]  

def openhtml(ddir,filename_tr):
    filename=ddir+filename_tr+".html"
    # dirty hack for character code
    try:
        with open(filename,"r") as fp:
            htmltext=fp.read()
    except:
        try:
            with open(filename,"r",encoding='euc_jp') as fp:
                htmltext=fp.read()
        except:
            try:
                with open(filename,"r",encoding='CP932') as fp:
                    htmltext=fp.read()
            except:
                print("cannot read"+filename)
    return htmltext
    
def souptable2csv(table,writedown=False,filename_tr=""):
    table_header=table.find_all("th")
    table_contents=table.find_all("tr")
    outdata  =get_tabledata(table_contents)
    outheader=get_tableheader(table_header)
    try:
        outdata[0]=outheader[0]
    except:
        print(filename_tr)
        print(table_header)
        
    if(writedown):
        with open(ddir+filename_tr+".csv", 'w') as fpw:
            writer = csv.writer(fpw, lineterminator='\n')
            writer.writerows(outdata)

def converthtml2csv(filename_tr,writedown=False):
    htmltext=openhtml(ddir,filename_tr)
    soup = BeautifulSoup(htmltext, "html.parser")
    table=soup #.find_all("table")
    souptable2csv(table,writedown,filename_tr)

In [39]:
import glob
htmlfiles=glob.glob("data/*/*.html")
#htmlfiles=glob.glob("data/?/*.html")

for f in htmlfiles:
    filename_tr=f[4:]
    filename_tr=filename_tr.split(".")[0]
    #print (filename_tr)
    converthtml2csv(filename_tr,True)

/spell/アンケート・文花帖クリアシーン数結果
[]
/10/第１０回東方シリーズ人気投票 投票者アンケート結果
[]
/5/第５回東方シリーズ人気投票 投票者アンケート結果
[]
/4/第４回東方シリーズ人気投票 投票者アンケート結果
[]
/4/第４回東方シリーズ人気投票 その他集計結果
[]
/4/第４回東方シリーズ人気投票 キャラ部門集計結果
[]
/4/第４回東方シリーズ人気投票 音楽部門集計結果
[]
/3/第３回東方シリーズ人気投票 その他集計結果
[]
/3/第３回東方シリーズ人気投票 音楽部門集計結果
[]
/3/第３回東方シリーズ人気投票 キャラ部門集計結果
[]
/3/第３回東方シリーズ人気投票 スペルカード部門集計結果
[]
/3/第３回東方シリーズ人気投票 投票者アンケート結果
[]
/2/第２回東方シリーズ人気投票 音楽部門結果
[]
/2/第２回東方シリーズ人気投票 キャラ部門結果
[]
/2/第２回東方シリーズ人気投票 その他集計結果
[]
/2/第２回東方シリーズ人気投票 弾幕部門結果
[]
/7/第７回東方シリーズ人気投票 投票者アンケート結果
[]
/11/経緯報告書 - 第11回 東方Project 人気投票
[]
/6/第６回東方シリーズ人気投票 投票者アンケート結果
[]


In [40]:
!wc data/*/*.csv

    76     84   1240 data/1/東方シリーズキャラ人気投票 [Vote Top].csv
   225    399   4956 data/1/東方シリーズ音楽人気投票 [Vote Top].csv
   225    399   4956 data/1/東方シリーズ音楽人気投票 [Vote Top]_iro.csv
   160    225   8124 data/10/第１０回東方シリーズ人気投票 キャラ部門集計結果.csv
   432    823  27956 data/10/第１０回東方シリーズ人気投票 音楽部門集計結果.csv
   216    228   5792 data/10/第１０回東方シリーズ人気投票 投票者アンケート結果.csv
   268    267  10151 data/11/アンケート投票結果 - 第11回 東方Project 人気投票.csv
   173    239   8701 data/11/キャラ部門投票結果 - 第11回 東方Project 人気投票.csv
   916   1308  26604 data/11/音楽部門投票結果 - 第11回 東方Project 人気投票.csv
     0      0      0 data/11/経緯報告書 - 第11回 東方Project 人気投票.csv
   260    259  11145 data/12/アンケート部門集計結果 - 第12回 東方Project 人気投票.csv
   476    858  27791 data/12/音楽部門投票結果 - 第12回 東方Project 人気投票.csv
    42     47   1888 data/12/作品部門投票結果 - 第12回 東方Project 人気投票.csv
   170    237   9271 data/12/人妖部門投票結果 - 第12回 東方Project 人気投票.csv
   274    273  11740 data/13/アンケート集計結果 - 第13回 東方Project 人気投票.csv
   438   1002  24687 data/13/ベストパートナー部門投票結果 - 第13回 東方Project 人気投票.csv
   4

In [41]:
import pandas as pd

第一回は特別な処理が必要になる

In [82]:
vote1_char=pd.read_csv("data/1/東方シリーズキャラ人気投票 [Vote Top].csv")
vote1_music=pd.read_csv("data/1/東方シリーズ音楽人気投票 [Vote Top].csv")

In [85]:
vote1_char=vote1_char.dropna(how="all")
vote1_char=vote1_char.dropna(thresh=5,axis=1)
vote1_char=vote1_char.dropna(thresh=2,axis=0)

In [86]:
vote1_char

,ランク,投票数,グラフ
0,十六夜咲夜,1位,95.0
1,霧雨 魔理沙,-,95.0
2,博麗 霊夢,3位,62.0
3,チルノ,4位,51.0
4,パチュリー・ノーレッジ,5位,46.0
5,橙,6位,43.0
6,フランドール・スカーレット,7位,32.0
7,レミリア・スカーレット,8位,28.0
8,アリス・マーガトロイド,9位,27.0
9,魅魔,10位,14.0


In [45]:
vote1_music

,ランク,投票,項目,投票数,投票理由,グラフ
0,亡き王女の為のセプテット,NaN,NaN,1位,NaN,81.0
1,U.N.オーエンは彼女なのか？,NaN,NaN,2位,NaN,77.0
2,明治十七年の上海アリス,NaN,NaN,3位,NaN,49.0
3,上海紅茶館 ～ Chinese Tea,NaN,NaN,4位,NaN,47.0
4,月時計 ～ ルナ・ダイアル,NaN,NaN,5位,NaN,45.0
...,...,...,...,...,...,...
218,NaN,NaN,NaN,NaN,NaN,NaN
219,夢想時空,NaN,NaN,NaN,NaN,NaN
220,NaN,NaN,NaN,NaN,NaN,NaN
221,妖々夢 ～ Snow or Cherry Petal,NaN,NaN,NaN,NaN,NaN
